## 1. Import Required Packages

In [2]:
import pandas as pd
import math
import os

os.chdir(r'C:/Users/이명아/Desktop/raw data')

## 2. Import Datasets

In [3]:
activity = pd.read_csv('train_activity.csv')
combat = pd.read_csv('train_combat.csv')
label = pd.read_csv('train_label.csv')
payment = pd.read_csv('train_payment.csv')
pledge = pd.read_csv('train_pledge.csv')
trade = pd.read_csv('train_trade.csv')

In [4]:
test1_activity = pd.read_csv('test1_activity.csv')
test1_combat = pd.read_csv('test1_combat.csv')
test1_payment = pd.read_csv('test1_payment.csv')
test1_pledge = pd.read_csv('test1_pledge.csv')
test1_trade = pd.read_csv('test1_trade.csv')

In [5]:
test2_activity = pd.read_csv('test2_activity.csv')
test2_combat = pd.read_csv('test2_combat.csv')
test2_payment = pd.read_csv('test2_payment.csv')
test2_pledge = pd.read_csv('test2_pledge.csv')
test2_trade = pd.read_csv('test2_trade.csv')

## 3. Combining_Datasets 
- activity data를 합쳐서 전체 day, acc_id, char_id, server를 뽑아냄.


- 여기에다가 계속 left_join할 예정

In [6]:
# 병합을 위한 데이터프레임 완성시키자. day, acc_id, char_id가 같아도 서버가 다른 경우도 있기 떄문에 server까지 포함시킨다
a1 = activity[['day','acc_id','char_id','server']]
a2 = test1_activity[['day','acc_id','char_id','server']]
a3 = test2_activity[['day','acc_id','char_id','server']]
total_df = pd.concat([a1,a2,a3]) 

In [6]:
total_df.shape

(3073650, 4)

In [7]:
total_df = total_df.sort_values(['acc_id','day']) ## acc_id와 day순으로 정렬해준다

In [8]:
total_df.drop_duplicates().shape ## 중복되는 항이 없다

(3073650, 4)

### 3_1. Label

In [9]:
label.head() ## payment에서 amount_spent변수가 겹치므로 라벨을 바꿔줍니다.

,acc_id,survival_time,amount_spent
0,27835,64,0.002559
1,12351,64,0.120154
2,125437,55,0.182593
3,104483,64,0.016241
4,4704,20,0.226396


In [10]:
label.columns = ['acc_id', 'survival_time', 'label_amount_spent']

In [11]:
label['acc_id'].nunique() ## 4만개 id

40000

In [12]:
total_df = pd.merge(total_df,label,how='left',on='acc_id') ## 앞에서 만들어놓은 total_df에 label데이터를 병함

In [13]:
total_df.shape

(3073650, 6)

In [14]:
total_df.head() # NaN은 testset임. 

,day,acc_id,char_id,server,survival_time,label_amount_spent
0,1,1,232344,ag,NaN,NaN
1,2,1,232344,ag,NaN,NaN
2,2,1,218648,ag,NaN,NaN
3,3,1,232344,ag,NaN,NaN
4,3,1,218648,ag,NaN,NaN


### 3_2. Activity

In [15]:
activity.head()

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,397380,aa,1.441844,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,1.10514,0.034006,0.0
1,1,75001,216231,aa,0.283219,2.247978,0.047085,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,-0.013963,0.0
2,1,75711,308769,aa,1.036910,2.956638,0.321821,0.166593,0.003470,1,0.245883,0.247337,0.0,0.0,0.00000,-0.001987,0.0
3,1,72230,387177,aa,0.229384,4.042102,0.099147,0.000000,0.001735,0,0.000000,0.000000,0.0,0.0,0.00000,-0.028884,0.0
4,1,34253,339862,aa,1.088405,0.596802,0.003377,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,0.011777,0.0


In [16]:
activity.sort_values(['char_id','day']).head(10) ## 같은 acc_id가 다양한 char_id를 가지고 있는 경우가 있다. 

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
121665,3,12365,2,ag,0.121714,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
702848,13,121046,3,as,0.126395,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
815776,15,121046,3,as,0.135758,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,-0.000020,0.0
984029,18,121046,3,as,0.044472,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1098536,20,121046,3,as,0.007022,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,-0.000003,0.0
1327873,24,121046,3,as,0.477494,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,-0.000020,0.0
1387084,25,121046,3,as,0.018725,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1509639,27,121046,3,as,1.172668,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.002467,0.0
1573080,28,121046,3,as,0.716240,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
88395,2,8885,5,at,0.004681,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [17]:
activity[['acc_id','char_id']].drop_duplicates().groupby('acc_id').count().reset_index()['char_id'].describe()
## 한사람이 평균 3.8개의 캐릭터. 

count    40000.000000
mean         3.812075
std          4.282028
min          1.000000
25%          1.000000
50%          3.000000
75%          5.000000
max        229.000000
Name: char_id, dtype: float64

In [26]:
total_activity = pd.concat([activity,test1_activity,test2_activity]) ## activity데이터를 합쳐서 total_df에 붙혀보자

In [27]:
total_df = pd.merge(total_df,total_activity,how='left',on=['acc_id','day','char_id','server'])

In [28]:
total_df.head(10)

,day,acc_id,char_id,server,survival_time,label_amount_spent,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,1,232344,ag,NaN,NaN,3.370544,2.259468,0.019597,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000266,0.0
1,2,1,232344,ag,NaN,NaN,3.340115,2.280420,0.019898,0.0,0.0,0,0.245883,0.0,0.0,0.0,0.0,-0.000230,0.0
2,2,1,218648,ag,NaN,NaN,0.004681,0.000000,0.000000,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,3,1,232344,ag,NaN,NaN,3.342456,2.241557,0.019653,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000154,0.0
4,3,1,218648,ag,NaN,NaN,0.002341,0.000000,0.000000,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
5,4,1,232344,ag,NaN,NaN,3.370544,2.215873,0.019488,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,-0.000105,0.0
6,5,1,232344,ag,NaN,NaN,3.361181,2.290896,0.020145,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000059,0.0
7,5,1,218648,ag,NaN,NaN,0.002341,0.000000,0.000000,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
8,6,1,232344,ag,NaN,NaN,3.337774,2.191880,0.019246,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,-0.000039,0.0
9,6,1,218648,ag,NaN,NaN,0.004681,0.000000,0.000000,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [29]:
total_df.shape ## 잘 병합되고 있다.

(3073650, 19)

### 3_3 Payment

In [30]:
payment.head()

,day,acc_id,amount_spent
0,18,27835,0.826123
1,23,27835,0.011735
2,27,27835,0.035204
3,17,12351,0.528062
4,11,125437,0.633674


In [31]:
payment['amount_spent'].describe()

count    66432.000000
mean         0.668198
std          0.909331
min          0.011735
25%          0.105612
50%          0.352041
75%          0.946990
max         11.733528
Name: amount_spent, dtype: float64

In [32]:
payment['acc_id'].nunique() ## pay를 안하면 기록이 없어서 activity에 비해 적다.

23726

In [33]:
total_payment = pd.concat([payment,test1_payment,test2_payment]) 

In [34]:
total_payment['acc_id'].nunique() ## 8만개의 아이디보다 적다. 반정도의 유저는 pay기록이 없는거

42679

In [35]:
total_df = pd.merge(total_df,total_payment,how='left',on=['acc_id','day'])

In [36]:
total_payment.head()

,day,acc_id,amount_spent
0,18,27835,0.826123
1,23,27835,0.011735
2,27,27835,0.035204
3,17,12351,0.528062
4,11,125437,0.633674


In [37]:
total_df.shape # 잘 병합되고 있다.

(3073650, 20)

### 3_4. Trade

In [38]:
trade.head(10)

,day,time,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price
0,7,21:13:05,1,ag,11439,385109,48152,34247,enchant_scroll,4.793968e-08,NaN
1,2,15:33:00,1,al,22220,259705,43228,379826,enchant_scroll,4.314571e-06,NaN
2,1,14:24:53,1,al,18212,164230,38145,34209,accessory,7.190952e-08,NaN
3,7,23:13:43,1,au,121622,49839,71226,281334,adena,9.587936e-03,NaN
4,2,12:43:33,0,bd,14421,344503,65220,210558,etc,1.198492e-07,0.000639
5,2,16:56:12,0,aj,90975,346488,22407,110658,enchant_scroll,7.190952e-08,0.005693
6,2,21:54:22,1,av,10578,286509,52677,365997,etc,4.793968e-05,NaN
7,7,14:14:46,0,bc,96563,209660,82644,54832,etc,1.198492e-07,0.067717
8,6,02:29:14,1,av,107975,5446,13885,181430,etc,7.670349e-06,NaN
9,6,02:29:02,0,as,91572,154600,41504,4697,etc,4.793968e-08,0.183391


In [39]:
print(trade['source_char_id'].nunique())
print(trade['target_char_id'].nunique())
print(trade['source_acc_id'].nunique())
print(trade['target_acc_id'].nunique())

## 거래데이터의 id는 라벨에 없는 애들이 있다. 

90413
109673
60645
72300


In [40]:
total_trade = pd.concat([trade,test1_trade,test2_trade]) 

In [41]:
total_trade.head()

,day,time,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price
0,7,21:13:05,1,ag,11439,385109,48152,34247,enchant_scroll,4.793968e-08,NaN
1,2,15:33:00,1,al,22220,259705,43228,379826,enchant_scroll,4.314571e-06,NaN
2,1,14:24:53,1,al,18212,164230,38145,34209,accessory,7.190952e-08,NaN
3,7,23:13:43,1,au,121622,49839,71226,281334,adena,9.587936e-03,NaN
4,2,12:43:33,0,bd,14421,344503,65220,210558,etc,1.198492e-07,0.000639


In [42]:
target_price_df = total_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()
target_price_df.columns = ['acc_id', 'char_id', 'day', 'server','target_accessory_price', 'target_adena_price', 'target_armor_price',
       'target_enchant_scroll_price', 'target_etc_price', 'target_spell_price', 'target_weapon_price']

In [87]:
# test1
test1_target_price_df = test1_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()
test1_target_price_df.columns = ['acc_id', 'char_id', 'day', 'server','target_accessory_price', 'target_adena_price', 'target_armor_price',
       'target_enchant_scroll_price', 'target_etc_price', 'target_spell_price', 'target_weapon_price']

In [11]:
# test2
test2_target_price_df = test2_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()
test2_target_price_df.columns = ['acc_id', 'char_id', 'day', 'server','target_accessory_price', 'target_adena_price', 'target_armor_price',
       'target_enchant_scroll_price', 'target_etc_price', 'target_spell_price', 'target_weapon_price']

In [43]:
target_amount_df = total_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

target_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'target_accessory_amount', 'target_adena_amount', 'target_armor_amount',
       'target_enchant_scroll_amount', 'target_etc_amount', 'target_spell_amount', 'target_weapon_amount']

In [90]:
# test1
test1_target_amount_df = test1_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

test1_target_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'target_accessory_amount', 'target_adena_amount', 'target_armor_amount',
       'target_enchant_scroll_amount', 'target_etc_amount', 'target_spell_amount', 'target_weapon_amount']

In [12]:
# test2
test2_target_amount_df = test2_trade.groupby(['target_acc_id','target_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['target_acc_id','target_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

test2_target_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'target_accessory_amount', 'target_adena_amount', 'target_armor_amount',
       'target_enchant_scroll_amount', 'target_etc_amount', 'target_spell_amount', 'target_weapon_amount']

In [44]:
source_price_df = total_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()

source_price_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_price', 'source_adena_price', 'source_armor_price',
       'source_enchant_scroll_price', 'source_etc_price', 'source_spell_price', 'source_weapon_price']

In [92]:
#test1
test1_source_price_df = test1_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()

test1_source_price_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_price', 'source_adena_price', 'source_armor_price',
       'source_enchant_scroll_price', 'source_etc_price', 'source_spell_price', 'source_weapon_price']

In [13]:
# test2
test2_source_price_df = test2_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_price'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_price').fillna(0).reset_index()

test2_source_price_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_price', 'source_adena_price', 'source_armor_price',
       'source_enchant_scroll_price', 'source_etc_price', 'source_spell_price', 'source_weapon_price']

In [45]:
source_amount_df = total_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

source_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_amount', 'source_adena_amount', 'source_armor_amount',
       'source_enchant_scroll_amount', 'source_etc_amount', 'source_spell_amount', 'source_weapon_amount']

In [94]:
# test1
test1_source_amount_df = test1_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

test1_source_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_amount', 'source_adena_amount', 'source_armor_amount',
       'source_enchant_scroll_amount', 'source_etc_amount', 'source_spell_amount', 'source_weapon_amount']

In [14]:
# test2
test2_source_amount_df = test2_trade.groupby(['source_acc_id','source_char_id','day','item_type','server'])['item_amount'].agg('sum').reset_index().pivot_table(index=['source_acc_id','source_char_id','day','server'],columns='item_type',values='item_amount').fillna(0).reset_index()

test2_source_amount_df.columns = ['acc_id', 'char_id', 'day','server', 'source_accessory_amount', 'source_adena_amount', 'source_armor_amount',
       'source_enchant_scroll_amount', 'source_etc_amount', 'source_spell_amount', 'source_weapon_amount']

In [46]:
total_df = pd.merge(total_df,target_price_df,how='left',on=['acc_id','day','char_id','server'])
total_df = pd.merge(total_df,target_amount_df,how='left',on=['acc_id','day','char_id','server'])
total_df = pd.merge(total_df,source_price_df,how='left',on=['acc_id','day','char_id','server'])
total_df = pd.merge(total_df,source_amount_df,how='left',on=['acc_id','day','char_id','server'])

In [47]:
total_df.columns

Index(['day', 'acc_id', 'char_id', 'server', 'survival_time',
       'label_amount_spent', 'playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'amount_spent', 'target_accessory_price', 'target_adena_price',
       'target_armor_price', 'target_enchant_scroll_price', 'target_etc_price',
       'target_spell_price', 'target_weapon_price', 'target_accessory_amount',
       'target_adena_amount', 'target_armor_amount',
       'target_enchant_scroll_amount', 'target_etc_amount',
       'target_spell_amount', 'target_weapon_amount', 'source_accessory_price',
       'source_adena_price', 'source_armor_price',
       'source_enchant_scroll_price', 'source_etc_price', 'source_spell_price',
       'source_weapon_price', 'source_accessory_amount', 'source_adena_amount',
       'source_armor_amount', 'source_enchant_scroll_amount',
       'source_etc_am

### 3_5. Pledge
- random_attacker_cnt, random_defender_cnt, same_pledge_cnt,temp_cnt,etc_cnt가 Combat이랑 겹친다. 앞에 Pledge를 붙여줘서 변수명을 바꿔준다 


- pledge_random_attacker_cnt, pledge_random_defender_cnt, pledge_same_pledge_cnt,pledge_temp_cnt,etc_cnt

In [48]:
pledge.head() 

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time
0,1,106660,345122,aa,381,0.036089,0.036089,0.0,0.0,0.0,0.0,0.000000,0.007078,0.027568,0.000000
1,1,46383,161773,aa,10892,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.006139
2,1,18495,321373,aa,13705,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000347
3,1,118293,366707,aa,2982,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001042
4,1,50421,158063,aa,27104,0.216536,0.072179,0.0,0.0,0.0,0.0,0.098883,0.056626,0.083978,0.000000


In [49]:
pledge.sort_values(['pledge_id','day']).head() ## 같은 day와 pledge_id의 경우 값들이 같다. 

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time
5557,1,39448,277729,ae,2,0.036089,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000232
840541,23,39448,277729,ae,2,0.036089,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000347
5734,1,65886,295854,ae,3,0.793965,0.50525,0.101991,1.194218,0.286125,0.0,0.494415,0.403461,0.832362,0.000000
5735,1,86844,180762,ae,3,0.793965,0.50525,0.101991,1.194218,0.286125,0.0,0.494415,0.403461,0.832362,0.000000
5736,1,26564,287697,ae,3,0.793965,0.50525,0.101991,1.194218,0.286125,0.0,0.494415,0.403461,0.832362,0.000000


In [50]:
total_pledge = pd.concat([pledge,test1_pledge,test2_pledge]) 

In [51]:
## 변수명 변경
total_pledge.columns = ['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'pledge_random_attacker_cnt',
       'pledge_random_defender_cnt', 'pledge_same_pledge_cnt', 'pledge_temp_cnt', 'pledge_etc_cnt',
       'combat_play_time', 'non_combat_play_time']

In [52]:
total_pledge.head()

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,pledge_random_attacker_cnt,pledge_random_defender_cnt,pledge_same_pledge_cnt,pledge_temp_cnt,pledge_etc_cnt,combat_play_time,non_combat_play_time
0,1,106660,345122,aa,381,0.036089,0.036089,0.0,0.0,0.0,0.0,0.000000,0.007078,0.027568,0.000000
1,1,46383,161773,aa,10892,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.006139
2,1,18495,321373,aa,13705,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000347
3,1,118293,366707,aa,2982,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001042
4,1,50421,158063,aa,27104,0.216536,0.072179,0.0,0.0,0.0,0.0,0.098883,0.056626,0.083978,0.000000


In [53]:
total_df.head()

,day,acc_id,char_id,server,survival_time,label_amount_spent,playtime,npc_kill,solo_exp,party_exp,...,source_etc_price,source_spell_price,source_weapon_price,source_accessory_amount,source_adena_amount,source_armor_amount,source_enchant_scroll_amount,source_etc_amount,source_spell_amount,source_weapon_amount
0,1,1,232344,ag,NaN,NaN,3.370544,2.259468,0.019597,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,232344,ag,NaN,NaN,3.340115,2.280420,0.019898,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,218648,ag,NaN,NaN,0.004681,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,232344,ag,NaN,NaN,3.342456,2.241557,0.019653,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1,218648,ag,NaN,NaN,0.002341,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
total_df = pd.merge(total_df,total_pledge,how='left',on=['day','acc_id','char_id','server'])

In [55]:
total_df.shape

(3073650, 59)

### 3_6. Combat

In [56]:
combat.sort_values(['char_id','day']).head()

,day,acc_id,char_id,server,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
1497291,3,12365,2,ag,2,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
463202,13,121046,3,as,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473782,15,121046,3,as,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489270,18,121046,3,as,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499708,20,121046,3,as,3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
total_combat = pd.concat([combat,test1_combat,test2_combat]) 

In [58]:
total_df = pd.merge(total_df,total_combat,how='left',on=['day','acc_id','char_id','server'])

In [59]:
total_df.head()

,day,acc_id,char_id,server,survival_time,label_amount_spent,playtime,npc_kill,solo_exp,party_exp,...,non_combat_play_time,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,1,1,232344,ag,NaN,NaN,3.370544,2.259468,0.019597,0.0,...,0.0,3.0,16.0,0.0,0.0,14.196774,0.0,0.0,1.801784,1.079416
1,2,1,232344,ag,NaN,NaN,3.340115,2.280420,0.019898,0.0,...,0.0,3.0,16.0,0.0,0.0,0.000000,0.0,0.0,2.927900,1.275674
2,2,1,218648,ag,NaN,NaN,0.004681,0.000000,0.000000,0.0,...,NaN,1.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,3,1,232344,ag,NaN,NaN,3.342456,2.241557,0.019653,0.0,...,0.0,3.0,16.0,0.0,0.0,0.000000,0.0,0.0,2.477454,0.883159
4,3,1,218648,ag,NaN,NaN,0.002341,0.000000,0.000000,0.0,...,NaN,1.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [60]:
total_df.columns

Index(['day', 'acc_id', 'char_id', 'server', 'survival_time',
       'label_amount_spent', 'playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'amount_spent', 'target_accessory_price', 'target_adena_price',
       'target_armor_price', 'target_enchant_scroll_price', 'target_etc_price',
       'target_spell_price', 'target_weapon_price', 'target_accessory_amount',
       'target_adena_amount', 'target_armor_amount',
       'target_enchant_scroll_amount', 'target_etc_amount',
       'target_spell_amount', 'target_weapon_amount', 'source_accessory_price',
       'source_adena_price', 'source_armor_price',
       'source_enchant_scroll_price', 'source_etc_price', 'source_spell_price',
       'source_weapon_price', 'source_accessory_amount', 'source_adena_amount',
       'source_armor_amount', 'source_enchant_scroll_amount',
       'source_etc_am

In [61]:
total_df.head()

,day,acc_id,char_id,server,survival_time,label_amount_spent,playtime,npc_kill,solo_exp,party_exp,...,non_combat_play_time,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,1,1,232344,ag,NaN,NaN,3.370544,2.259468,0.019597,0.0,...,0.0,3.0,16.0,0.0,0.0,14.196774,0.0,0.0,1.801784,1.079416
1,2,1,232344,ag,NaN,NaN,3.340115,2.280420,0.019898,0.0,...,0.0,3.0,16.0,0.0,0.0,0.000000,0.0,0.0,2.927900,1.275674
2,2,1,218648,ag,NaN,NaN,0.004681,0.000000,0.000000,0.0,...,NaN,1.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,3,1,232344,ag,NaN,NaN,3.342456,2.241557,0.019653,0.0,...,0.0,3.0,16.0,0.0,0.0,0.000000,0.0,0.0,2.477454,0.883159
4,3,1,218648,ag,NaN,NaN,0.002341,0.000000,0.000000,0.0,...,NaN,1.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


## 4. 전체데이터에서 train data분리

In [62]:
def notnan(x):
    if math.isnan(x):
        return False
    else:
        return True

In [63]:
train_total = total_df[total_df['survival_time'].apply(lambda x: notnan(x))] # label이 있는 데이터가 training data이다.

In [64]:
train_total['acc_id'].nunique()

40000

In [65]:
total_df.to_csv('total_df.csv')

In [66]:
train_total.to_csv('train_df.csv')

## Test data 분리

#### test1 분리

In [162]:
test1_df = a2

In [163]:
test1_df = pd.merge(test1_df, test1_activity, how ='left', on=['day','acc_id','char_id','server'])

In [164]:
test1_df = pd.merge(test1_df, test1_payment, how = 'left', on = ['day','acc_id'])

In [165]:
test1_df = pd.merge(test1_df, test1_target_price_df, how='left', on = ['acc_id','char_id','day','server'])
test1_df = pd.merge(test1_df, test1_target_amount_df, how='left', on = ['acc_id','char_id','day','server'])
test1_df = pd.merge(test1_df, test1_source_price_df, how='left', on = ['acc_id','char_id','day','server'])
test1_df = pd.merge(test1_df, test1_source_amount_df, how='left', on = ['acc_id','char_id','day','server'])

In [166]:
## 변수명 변경
test1_pledge.columns = ['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'pledge_random_attacker_cnt',
       'pledge_random_defender_cnt', 'pledge_same_pledge_cnt', 'pledge_temp_cnt', 'pledge_etc_cnt',
       'combat_play_time', 'non_combat_play_time']

In [167]:
test1_df = pd.merge(test1_df, test1_pledge, how ='left', on = ['day','acc_id','char_id','server'])

In [168]:
test1_df = pd.merge(test1_df, test1_combat, how = 'left', on = ['day','acc_id','char_id','server'])

In [171]:
test1_df.shape

(709928, 66)

In [172]:
test1_df['acc_id'].nunique()

20000

In [174]:
test1_df.to_csv("test1_df.csv")

####  test2 분리

In [7]:
test2_df = a3
test2_df.head()

,day,acc_id,char_id,server
0,1,43337,22671,aa
1,1,16892,260714,aa
2,1,13584,143979,aa
3,1,13584,382845,aa
4,1,13584,364142,aa


In [8]:
test2_df = pd.merge(test2_df, test2_activity, how = 'left', on = ['day','acc_id','char_id','server'])

In [9]:
test2_df = pd.merge(test2_df, test2_payment , how = 'left', on = ['day','acc_id'])

In [15]:
test2_df = pd.merge(test2_df, test2_target_price_df, how='left', on = ['acc_id','char_id','day','server'])
test2_df = pd.merge(test2_df, test2_target_amount_df, how='left', on = ['acc_id','char_id','day','server'])
test2_df = pd.merge(test2_df, test2_source_price_df, how='left', on = ['acc_id','char_id','day','server'])
test2_df = pd.merge(test2_df, test2_source_amount_df, how='left', on = ['acc_id','char_id','day','server'])

In [16]:
## 변수명 변경
test2_pledge.columns = ['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'pledge_random_attacker_cnt',
       'pledge_random_defender_cnt', 'pledge_same_pledge_cnt', 'pledge_temp_cnt', 'pledge_etc_cnt',
       'combat_play_time', 'non_combat_play_time']

In [17]:
test2_df = pd.merge(test2_df, test2_pledge, how = 'left', on = ['day','acc_id','char_id','server'])

In [18]:
test2_df = pd.merge(test2_df, test2_combat, how = 'left', on=['day','acc_id','char_id','server'])

In [19]:
test2_df.shape

(761888, 66)

In [20]:
test2_df['acc_id'].nunique()

20000

In [22]:
test2_df.to_csv("test2_df.csv")